# Yelp API - Lab


## Introduction 

Now that we've seen how the Yelp API works and some basic Folium visualizations, it's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [3]:
#Your code here
import requests

api_key = 'H7kuPWsJX1o-BnPZ_EtY_FXEH8TYFvjR29lrpYfozz4dPLPYj6FTc-BBPLRJJrHjwGHisiRfIgxEnOJXOa05negyuTBGPyil9R1FaYkW5mVbSdfutMSCkwqeDXXcXHYx'#put your api key (as a string) here
url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'location': 'NYC',
                'term': 'pizza'
            }
response = requests.get(url, headers=headers, params=url_params)

In [7]:
data = response.json()
data

{'businesses': [{'id': 'ysqgdbSrezXgVwER2kQWKA',
   'alias': 'julianas-pizza-brooklyn-5',
   'name': "Juliana's Pizza",
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/7JtwTxhWHf3YS70Ss_CfxA/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/julianas-pizza-brooklyn-5?adjust_creative=hhCG4KzuqdlB8l6SM4Chbg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=hhCG4KzuqdlB8l6SM4Chbg',
   'review_count': 1957,
   'categories': [{'alias': 'pizza', 'title': 'Pizza'}],
   'rating': 4.5,
   'coordinates': {'latitude': 40.7026153030093,
    'longitude': -73.9934159993549},
   'transactions': [],
   'price': '$$',
   'location': {'address1': '19 Old Fulton St',
    'address2': '',
    'address3': '',
    'city': 'Brooklyn',
    'zip_code': '11201',
    'country': 'US',
    'state': 'NY',
    'display_address': ['19 Old Fulton St', 'Brooklyn, NY 11201']},
   'phone': '+17185966700',
   'display_phone': '(718) 596-6700',
   'distance': 323.20506308227306},

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [8]:
# Your code here; use a function or loop to retrieve all the results from your original request
data.keys()

dict_keys(['businesses', 'total', 'region'])

In [19]:
data['region']

{'center': {'longitude': -73.99429321289062, 'latitude': 40.70544486444615}}

## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [34]:
#Your code here
import pandas as pd
df = pd.DataFrame.from_dict(data['businesses'])

In [31]:
data['businesses'][0]['coordinates']

{'latitude': 40.7026153030093, 'longitude': -73.9934159993549}

In [37]:
df['coordinates'][0]['latitude']



40.7026153030093

In [43]:
keys = df.coordinates.iloc[0].keys() #Get dictionary keys
#Keep track of columns we make for subsequent preview
new_cols = []
#Create a new feature for each of these keys
for key in keys:
  new_col = 'coordinates_{}'.format(key) #Create new column name
  df[new_col] = df.coordinates.map(lambda x: x[key]) #Create a new column
  new_cols.append(new_col)
df[new_cols]

df.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url,coordinates_latitude,coordinates_longitude
0,julianas-pizza-brooklyn-5,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.7026153030093, 'longitude': -7...",(718) 596-6700,323.205063,ysqgdbSrezXgVwER2kQWKA,https://s3-media1.fl.yelpcdn.com/bphoto/7JtwTx...,False,"{'address1': '19 Old Fulton St', 'address2': '...",Juliana's Pizza,+17185966700,$$,4.5,1957,[],https://www.yelp.com/biz/julianas-pizza-brookl...,40.702615,-73.993416
1,prince-street-pizza-new-york-2,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.7229087705545, 'longitude': -7...",(212) 966-4100,1941.963817,zj8Lq1T8KIC5zwFief15jg,https://s3-media3.fl.yelpcdn.com/bphoto/ZAukOy...,False,"{'address1': '27 Prince St', 'address2': None,...",Prince Street Pizza,+12129664100,$,4.5,2746,"[pickup, delivery]",https://www.yelp.com/biz/prince-street-pizza-n...,40.722909,-73.994486
2,lombardis-pizza-new-york-4,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.7215934960083, 'longitude': -7...",(212) 941-7994,1798.995978,WIhm0W9197f_rRtDziq5qQ,https://s3-media1.fl.yelpcdn.com/bphoto/UZ6V_h...,False,"{'address1': '32 Spring St', 'address2': '', '...",Lombardi's Pizza,+12129417994,$$,4.0,5843,"[pickup, delivery]",https://www.yelp.com/biz/lombardis-pizza-new-y...,40.721593,-73.995596
3,l-industrie-pizzeria-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.71162, 'longitude': -73.95783}",(718) 599-0002,3145.016041,v1DHGRNCH9247WLYoaoA9A,https://s3-media2.fl.yelpcdn.com/bphoto/SRL_Rz...,False,"{'address1': '254 S 2nd St', 'address2': '', '...",L'industrie Pizzeria,+17185990002,$,5.0,416,"[pickup, delivery]",https://www.yelp.com/biz/l-industrie-pizzeria-...,40.711620,-73.957830
4,rizzos-fine-pizza-new-york-4,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.720852, 'longitude': -73.984456}",(646) 454-1262,1903.278835,nUpA3sY5b6dynC7aj0eV6A,https://s3-media3.fl.yelpcdn.com/bphoto/r4OPoX...,False,"{'address1': '17 Clinton St', 'address2': '', ...",Rizzo's Fine Pizza,+16464541262,$$,4.5,684,"[restaurant_reservation, pickup, delivery]",https://www.yelp.com/biz/rizzos-fine-pizza-new...,40.720852,-73.984456


## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [25]:
#Your code here
! pip install folium
import folium


In [33]:
for i in data['businesses']:
    print(i)
    

{'id': 'ysqgdbSrezXgVwER2kQWKA', 'alias': 'julianas-pizza-brooklyn-5', 'name': "Juliana's Pizza", 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/7JtwTxhWHf3YS70Ss_CfxA/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/julianas-pizza-brooklyn-5?adjust_creative=hhCG4KzuqdlB8l6SM4Chbg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=hhCG4KzuqdlB8l6SM4Chbg', 'review_count': 1957, 'categories': [{'alias': 'pizza', 'title': 'Pizza'}], 'rating': 4.5, 'coordinates': {'latitude': 40.7026153030093, 'longitude': -73.9934159993549}, 'transactions': [], 'price': '$$', 'location': {'address1': '19 Old Fulton St', 'address2': '', 'address3': '', 'city': 'Brooklyn', 'zip_code': '11201', 'country': 'US', 'state': 'NY', 'display_address': ['19 Old Fulton St', 'Brooklyn, NY 11201']}, 'phone': '+17185966700', 'display_phone': '(718) 596-6700', 'distance': 323.20506308227306}
{'id': 'zj8Lq1T8KIC5zwFief15jg', 'alias': 'prince-street-pizza-new-york-2', 'name': 'Prince S

In [47]:
df['coordinates_latitude'].iloc[0]

40.7026153030093

In [57]:
lat = 40.70
long = -73.99
# {'longitude': -73.99429321289062, 'latitude': 40.70544486444615}

#Create a map of the area
base_map = folium.Map([lat, long], zoom_start=12)
base_map

In [58]:
for i in range(len(df)-1):
    lat = df['coordinates_latitude'].iloc[i]
    long = df['coordinates_longitude'].iloc[i]
    marker = folium.Marker(location=[lat, long])
    marker.add_to(base_map)
base_map

In [60]:
for i in range(len(df)-1):
    restaurant = df['alias'].iloc[i]
    lat = df['coordinates_latitude'].iloc[i]
    long = df['coordinates_longitude'].iloc[i]
    popup_text = "Restaurant: {}, Latitude: {}, Longitude: {}".format(restaurant,lat,long)
    popup = folium.Popup(popup_text, parse_html=True)
    marker = folium.Marker(location=[lat, long], popup=popup)
    marker.add_to(base_map)
base_map

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!